# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4088494919
date='September 20, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
sgmt

,Name,Time,Speed,Power,Info
0,Puente 77 Sur - Variante 0.75km 29m 3.8%,3:12,14.1km/h,279W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
1,Union - Aries 0.54km 26m 4.4%,2:30,13.1km/h,308W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
2,Unicornio - Monaco 2.28km 85m 3.1%,9:14,14.8km/h,254W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
3,unicornio ancon 1.13km 63m 5.4%,5:18,12.9km/h,280W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
4,Sprint Largo (El ciclista) 0.61km 22m 3.3%,2:51,13.0km/h,260W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
5,Sprint (El ciclista) 0.34km 17m 5.1%,1:37,12.7km/h,314W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
6,Ancón Sur - Caldas 6.22km 88m 1.4%,19:47,18.9km/h,232W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
7,La Estrella Climb 0.86km 33m 3.8%,3:08,16.6km/h,315W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
8,Inmaculada - Caldas 4.37km 44m 1.0%,13:34,19.4km/h,219W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."
9,Alto de Gazu - Pagaucho 0.49km 25m 5.1%,2:04,14.2km/h,252W,"{'Calories': {'Avg': '3901', 'Max': '3901'}, '..."


In [6]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [7]:
df.to_json(db_file)

### Save notebook here

In [8]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [9]:
!bash commit.sh

[master c6be34f] Morning Ride 8:25 AM on Sunday, October 25, 2020
 2 files changed, 474 insertions(+), 289 deletions(-)
 rewrite db/strava.json (68%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 24.87 KiB | 2.26 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   fc62460..c6be34f  master -> master


In [10]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
